# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
#Your code here
import pandas as pd
import numpy as np

df = pd.read_csv("homepage_actions.csv")
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [3]:
df.describe()

,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [4]:
df.action.value_counts()

action
view     6328
click    1860
Name: count, dtype: int64

In [5]:
click = set(df[df.action=='click']['id'].unique())
view = set(df[df.action=='view']['id'].unique())
print("Number of Viewers who didn't click: {}".format(len(view-click)))
print("Number of Clickers who didn't view: {}".format(len(click-view)))

Number of Viewers who didn't click: 4468
Number of Clickers who didn't view: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [6]:
df[df.group == 'experiment']
df[df.group == 'control']


,timestamp,id,group,action
3,2016-09-24 19:59:02.646620,671993,control,view
9,2016-09-24 21:00:12.278374,560027,control,view
23,2016-09-25 00:25:14.141290,281985,control,view
24,2016-09-25 01:14:48.603202,407864,control,view
25,2016-09-25 02:16:11.046654,342984,control,view
...,...,...,...,...
8178,2017-01-18 08:17:12.675797,616692,control,view
8181,2017-01-18 09:07:37.661143,795585,control,view
8182,2017-01-18 09:09:17.363917,795585,control,click
8186,2017-01-18 10:08:51.588469,505451,control,view


In [7]:
#Your code here
# check for overlap between columns A in df1 and df2
overlap = df[df.group == 'experiment'].isin(df[df.group == 'control'])
print(overlap)
# This means there is no overlap.

      timestamp     id  group  action
0         False  False  False   False
1         False  False  False   False
2         False  False  False   False
4         False  False  False   False
5         False  False  False   False
...         ...    ...    ...     ...
8179      False  False  False   False
8180      False  False  False   False
8183      False  False  False   False
8184      False  False  False   False
8185      False  False  False   False

[3924 rows x 4 columns]


In [8]:
# Null Hypothesis: There was no difference in the effect of the experiment homepage and control homepage.
# Alternative hypothesis: The experiment homepage was more effective than control homepage.
# Signficance level : 0.05.

In [12]:
df['count'] = 1
df.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [13]:
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)



print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), len(experiment)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control.click.mean(), experiment.click.mean()))
control.head()

Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932.0	Experiment: 928.0
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


In [14]:
def welch_t(a, b):
    
    """ Calculate Welch's t-statistic for two samples. """
    a_mean = a.mean()
    b_mean = b.mean()
    var_a,var_b = np.var(a, ddof=1),np.var(b, ddof=1)
    std = np.sqrt(var_a/len(a)+(var_b/len(b)))
    t_score = (a_mean - b_mean)/std

    
    return np.abs(t_score)# Return the t-score!

welch_t(control, experiment)

action
click    2.61544
view         NaN
dtype: float64

In [15]:
def welch_df(a, b):
    
    """ Calculate the effective degrees of freedom for two samples. """
    var_a,var_b = np.var(a, ddof=1),np.var(b, ddof=1)
    a1 = a.size
    b1 = b.size
    num = (var_a/a1+(var_b/b1))**2
    denom = (var_a/a1)**2/(a1-1) +(var_b/b1)**2/(b1-1)

    

    return num/denom# Return the degrees of freedom

welch_df(control, experiment)

action
click    12424.290363
view              NaN
dtype: float64

In [17]:
t = welch_t(control, experiment)
df = welch_df(control, experiment)
print(t.click, df.click)

2.615440020788211 12424.290363328171


In [18]:
import scipy.stats as stats

p = 1 - stats.t.cdf(t.click, df.click)
print(p)

0.004461013445251516


In [ ]:
# There is a 0.44 percent probability that the experimental homepage had a bigger effect than a control homepage. In that the p-value is lesser than
# the significance level there is reason to reject null hypothesis and accept alternative hypothesis.
# This means the experiment homepage does has a bigger effect than control homepage.

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [20]:
#Your code here
control.click.mean()
expected_no_of_experimentclicks = control.click.mean() * len(experiment)
expected_no_of_experimentclicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [22]:
#Your code here
p = control.click.mean()
n = len(experiment)
var = n *p*(1-p)
std = np.sqrt(var)
std#actual number of stds is experiment had the same click through rate as control group.

24.568547907005815

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [24]:
#Your code here
x_bar = experiment.click.sum()
mu_0 = expected_no_of_experimentclicks
z = (x_bar - mu_0)/(std)
z
p_val = stats.norm.sf(z) #or 1 - stats.norm.cdf(z_score)
print(p_val)

0.00012486528006951198


In [ ]:
# The p-value is slightly lower but it still upholds the conclusion that the experiment homepage has more effect than control homepage.
# This p-value still indicates rejection of the null hypothesis.

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: ** The p-value is slightly lower but it still upholds the conclusion that the experiment homepage has more effect than control homepage.
 This p-value still indicates rejection of the null hypothesis.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.